In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [8]:
ratings = pd.read_csv("C:\\share\\data\\ml-1m\\ratings.dat",
                      sep="::",
                      header=None,
                      engine='python',
                      names=["UserID", "MovieID", "Rating", "Timestamp"])
ratings.dropna(axis=0, inplace=True)
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [62]:
ratings2 = pd.DataFrame([[0]], columns=['temp'])
movie_names = set()
for (idx, row), idxlimit in zip(ratings.iterrows(), range(10000000)):
    movie_names.add(row['MovieID'])
movie_names_sort = sorted(list(movie_names))
ratings2 = pd.DataFrame([], columns=movie_names_sort)
# ratings2

In [63]:
user_names = set()
for (idx, row), idxlimit in zip(ratings.iterrows(), range(100000000)):
    user_names.add(row['UserID'])
user_names_sort = sorted(list(user_names))
user_names_pd = pd.DataFrame(user_names_sort, columns=['UserID'])
ratings2['UserID'] = user_names_pd['UserID']
ratings2.fillna(3, inplace=True)
# ratings2

C:\Users\wangy\AppData\Local\Temp\ipykernel_15932\2131049677.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings2.fillna(3, inplace=True)


In [64]:
ratings3 = ratings2.copy()
ratings3.index = ratings3['UserID']
ratings3.drop(columns=['UserID'], inplace=True)
for (idx, row), idxlimit in zip(ratings.iterrows(), range(100000000)):
    ratings3.loc[row['UserID'], row['MovieID']] = row['Rating']
ratings3 = (ratings3 - 1) / 4
# ratings3

In [27]:
ratings3.to_csv("../temp/ratings3.csv")

In [65]:
from sklearn.decomposition import TruncatedSVD

model = TruncatedSVD(n_components=30)
model.fit(ratings3)
ratings3_svd = model.transform(ratings3)
ratings3_svd

array([[ 3.07042091e+01, -3.58647231e-01, -9.59363386e-02, ...,
         1.13333372e-01, -1.00190251e-01,  7.20140551e-02],
       [ 3.08362333e+01, -6.53985019e-02, -3.50826069e-01, ...,
         1.49949454e-01, -2.33908902e-01,  1.33022886e-01],
       [ 3.06314495e+01, -4.14879342e-01, -2.60641762e-01, ...,
        -1.43241880e-01, -2.00444112e-01, -2.31777617e-01],
       ...,
       [ 3.04907130e+01, -8.70982163e-01,  1.42245562e-01, ...,
        -4.45846346e-02,  2.72106349e-02, -7.00095153e-02],
       [ 3.08929010e+01, -1.55005346e-01,  6.50729825e-01, ...,
         8.54186594e-02,  1.14769896e-01,  6.90275939e-02],
       [ 3.12949274e+01,  1.00265635e+00,  1.96111912e+00, ...,
        -5.08309618e-01,  5.26662509e-02, -1.18827124e-01]])

In [ ]:
ratings4 = model.inverse_transform(ratings3_svd)
ratings4 = pd.DataFrame(ratings4,
                        columns=ratings3.columns,
                        index=ratings3.index)
ratings4

In [67]:
result_pd = pd.DataFrame([], columns=['rec'], index=ratings3.index)
test_data = None
for (idx, row), (idx2, row2), i in zip(ratings4.iterrows(), ratings3.iterrows(), range(1000)):
    # print(row[row2==0.5].sort_values(ascending=False))
    test_data = row[row2 == 0.5].sort_values(ascending=False)
    result_pd.loc[idx, 'rec'] = ",".join(
        [str(iii) + ":" + str(rrr)[:5] for iii, rrr, jjj in zip(test_data.index, test_data, range(10))])
result_pd

,rec
UserID,
1,"364:0.721,318:0.689,34:0.681,2081:0.675,356:0.647,593:0.644,2080:0.634,1282:0.625,2078:0.624,3751:0.615"
2,"527:0.809,1704:0.726,260:0.695,150:0.693,1961:0.692,1580:0.680,2762:0.679,1097:0.659,780:0.651,1250:0.648"
3,"110:0.661,2628:0.654,2804:0.621,2918:0.620,1270:0.607,1220:0.605,1278:0.603,2716:0.603,2000:0.601,1073:0.597"
4,"858:0.651,589:0.635,1210:0.628,110:0.619,1200:0.615,457:0.610,1291:0.607,1304:0.601,1610:0.597,2571:0.596"
5,"1617:0.679,2762:0.661,3160:0.631,223:0.627,1148:0.618,1673:0.608,1358:0.606,3481:0.605,293:0.598,745:0.596"
...,...
6036,NaN
6037,NaN
6038,NaN


In [68]:
result_pd.to_csv("../temp/result.csv")